In [2]:
%load_ext watermark

In [8]:
import pymc as pm
import pandas as pd
import itertools as itt
import arviz as az
from pathlib import Path

TRACE_PATH = '../traces/'

%watermark --iversions

arviz : 0.12.0
pandas: 1.4.2
pymc  : 4.0.0b6



In [22]:
traces = {p.stem : {"path": str(p.absolute())} for p in Path(TRACE_PATH).glob("*.netcdf")}

In [27]:
for t in traces.values():
    t['trace'] = az.from_netcdf(t['path'])
    t['summary'] = az.summary(t['trace'], hdi_prob=0.95)

In [29]:
for name, trace in traces.items():
    print(name)
    print(trace['summary'].loc[["cup_impact[0, 0]", "regular_advantage[0, 0]"], :])

single_flat_model
                          mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  \
cup_impact[0, 0]        -1.336  2.451    -5.838      0.847      0.138   
regular_advantage[0, 0]  2.093  1.025     0.329      4.056      0.074   

                         mcse_sd  ess_bulk  ess_tail  r_hat  
cup_impact[0, 0]           0.137     314.0     776.0   1.02  
regular_advantage[0, 0]    0.054     240.0     620.0   1.02  
single_empirical_model
                          mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  \
cup_impact[0, 0]        -0.208  0.923    -1.686      1.085      0.098   
regular_advantage[0, 0]  2.651  1.104     0.613      4.176      0.135   

                         mcse_sd  ess_bulk  ess_tail  r_hat  
cup_impact[0, 0]           0.069      63.0    1539.0   1.04  
regular_advantage[0, 0]    0.118      47.0     868.0   1.06  
team_flat_model
                          mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  \
cup_impact[0, 0]         0.015  0.399    -0.701      0.807

In [31]:
az.compare({n : t['trace'] for n,t in traces.items()}, scale='deviance')

/home/aurimas/apps/anaconda3/envs/pymc-dev-py39/lib/python3.9/site-packages/arviz/stats/stats.py:1048: RuntimeWarning: overflow encountered in exp
  weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)
/home/aurimas/apps/anaconda3/envs/pymc-dev-py39/lib/python3.9/site-packages/numpy/core/_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/aurimas/apps/anaconda3/envs/pymc-dev-py39/lib/python3.9/site-packages/arviz/stats/stats.py:811: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
/home/aurimas/apps/anaconda3/envs/pymc-dev-py39/lib/python3.9/site-packages/arvi

,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
team_empirical_model,0,1514.618287,29.783470,0.000000,8.777643e-01,19.059904,0.000000,True,deviance
single_empirical_model,1,1527.439632,26.555048,12.821345,5.394872e-14,18.812307,3.845026,True,deviance
single_flat_model,2,1536.528002,37.393995,21.909715,3.061349e-14,19.521192,9.794799,True,deviance
team_flat_model,3,1536.980015,33.206531,22.361728,1.222357e-01,21.415955,11.132313,True,deviance
